<a href="https://colab.research.google.com/github/AtilioFPedroni/Pc__Gillespie/blob/main/PC_Gelispe_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
!pip install -q networkx matplotlib numpy pandas

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

rng = np.random.default_rng()

In [11]:
a = [15,578,8987]

retirou = a.pop(1)

retirou, a

(578, [15, 8987])

In [66]:
def processo_contato(g, lista_inicial_de_infectados, beta, mu, tf):
    
    # condicao inicial
    infectados = lista_inicial_de_infectados.copy()

    estados = {vertice: 0 for vertice in g.nodes()}
    
    for infectado in infectados:
        estados[infectado] = 1

    t = [0.0]
    n_infectados = [len(infectados)]

    n_infectados_atual = len(infectados)

    while t[-1] <= tf:

        if n_infectados_atual == 0:
            break

        taxa_total = beta * len(infectados) + mu * len(infectados)

        dt = -np.log(1.0 - rng.random()) / taxa_total

        # definir qual processo vai acontecer
        z = rng.random()
        if (z < mu*len(infectados) / taxa_total):
            # acontecer um processo de cura

            posicao_do_infectado = rng.integers(0, len(infectados)) # escolhe um infectado ao acaso, na posicao da lista
            infectado = infectados.pop(posicao_do_infectado) # retirar o infectado da lista

            estados[infectado] = 0

            n_infectados_atual -= 1

        else: # é equivalente a fazer elseif (z < beta*len(infectados) / taxa_total + mu*len(infectados) / taxa_total)
            # vai acontecer um processo de infecção

            # escolher um infectado ao acaso
            posicao_do_infectado = rng.integers(0, len(infectados)) # escolhe um infectado ao acaso, na posicao da lista

            infectado = infectados[posicao_do_infectado]

            # escolher um vizinho ao acaso
            vizinho = rng.choice(list(g.neighbors(infectado))) # escolher um vizinho ao acaso

            if estados[vizinho] == 0:
                infectados.append(vizinho) # adiciona ele à lista
                estados[vizinho] = 1

                n_infectados_atual += 1
            # caso contrário, nada é feito de diferente

        t.append(t[-1] + dt)
        n_infectados.append(n_infectados_atual)

    return t, n_infectados

In [81]:
#%%timeit -n 1 -r 1
def simulacao_1D(n, betas, mu=1.0, tf = 100, ni=1, n_amostras=1):
    results = {}

    G = nx.circulant_graph(n, [1])

    for beta in betas:
        results[beta] = []
        for id_da_amostra in range(n_amostras):
            lista_de_infectados_inicial = rng.choice(G.nodes(), size=ni).tolist()
            t, n_infectados = processo_contato(G, lista_de_infectados_inicial, beta, mu, tf)

            df = pd.DataFrame({'t': t, 'n_infectados': n_infectados})
            df['beta'] = beta
            df['id_da_amostra'] = id_da_amostra

            results[beta].append(df)
        results[beta] = pd.concat(results[beta])
    results = pd.concat(results.values())
    return results

######################################  Parâmetros ##############################

n = 100
betas = [0.1, 1, 1.5, 3.29, 5]
mu = 1.0
tf = 100
ni = 1

results = simulacao_1D(n, betas, mu, tf, ni, n_amostras=1)

results

{0.1: [          t  n_infectados  beta  id_da_amostra
  0  0.000000             1   0.1              0
  1  0.045513             2   0.1              0
  2  0.434174             1   0.1              0
  3  0.671419             0   0.1              0],
 1: [          t  n_infectados  beta  id_da_amostra
  0  0.000000             1     1              0
  1  0.344434             0     1              0],
 1.5: [           t  n_infectados  beta  id_da_amostra
  0   0.000000             1   1.5              0
  1   0.031176             2   1.5              0
  2   0.214417             3   1.5              0
  3   0.330407             2   1.5              0
  4   0.790037             2   1.5              0
  5   1.030426             3   1.5              0
  6   1.084215             2   1.5              0
  7   1.499582             1   1.5              0
  8   2.337474             2   1.5              0
  9   2.871984             3   1.5              0
  10  2.966905             3   1.5       

In [83]:
results[0.1][0]

,t,n_infectados,beta,id_da_amostra
0,0.000000,1,0.1,0
1,0.045513,2,0.1,0
2,0.434174,1,0.1,0
3,0.671419,0,0.1,0


In [74]:
# dataframe com as colunas beta, amostra, t, n_infectados
import pandas as pd

df = pd.DataFrame(columns=['beta', 'amostra', 't', 'n_infectados'])

for beta, lista_de_resultados in results.items():
    for id_da_amostra, (t, n_infectados) in enumerate(lista_de_resultados):
        df = df.append(pd.DataFrame({'beta': beta, 'amostra': id_da_amostra, 't': t, 'n_infectados': n_infectados}))
df

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
# melhor usar o pandas
import pandas as pd

